In [7]:
%load_ext autoreload
%autoreload 2

from algebra import *
from cache import *
from costs import *
from features import *
from gradients import *
from helpers import *
from model import *
from splits import *

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import csv
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
SUB_SAMPLE = True
CACHE_DIR = "test/cache/" if SUB_SAMPLE else "cache/"
SUBMISSIONS_DIR = "test/submissions/" if SUB_SAMPLE else "submissions/"

In [9]:
y, x, ids = load_csv_data('data/train.csv', SUB_SAMPLE)

# 1 - Analytical Results

### Ridge Regression with Fixed Degree

##### Without Validation

In [20]:
def clean_standardize_expand(y, x, h):
        
    degree = int(h['degree'])

    x = remove_errors(x)
    x = remove_outliers(x)
    x = standardize_all(x)
    x = remove_nan_features(x)
    x = build_poly(x, degree)

    return y, x

In [ ]:
def ridge_regression_analytical(y, x, h):

    lambda_ = float(h['lambda'])
    degree = int(h['degree'])

    w = ridge_regression(y, x, lambda_)
    
    return {
        'w': w,
        'mse': compute_mse(y, x, w)
    }

In [ ]:
hs = { 
    'degree': [5, 6, 7], 
    'lambda': 1e-4,
}

_ = evaluate(
    clean = clean_standardize_expand, 
    fit   = ridge_regression_analytical, 
    x     = x, 
    y     = y, 
    hs    = hs, 
    cache = CACHE_DIR + 'clean_standardize_expand_ridge_regression_analytical'
)

##### Using Cross-Validation

Here, we implement the same model with cross-validation.

In [ ]:
hs = { 
    'degree': np.arange(4, 16), 
    'lambda': np.logspace(-8, -4, 5),
    'k_fold': 4,
    'seed': 0
}

def mse(y, x, w):
    return { 'mse' : compute_mse(y, x, w) }



evaluate(
    clean = cross_validate(ridge_regression_analytical, mse), 
    fit   = fit_function, 
    x     = x,
    y     = y, 
    hs    = hs, 
    cache = CACHE_DIR + 'clean_standardize_expand_cross_validate_ridge_regression_analytical_mse'
)

In [ ]:
# myModel.predict(best_h, x, y, SUBMISSIONS_DIR + 'RidgeRegression_MSE_FixedDegree_CrossValidation_Model')

## Gradient Descents

#### Least Square

#### Ridge Regression

#### Lasso

# Logistic Regression

In [18]:
def map_logistic(clean):
    
    def inner_function(y, x, h):
        y, x = clean(y, x, h)
        y = np.where(y == 1, 1, 0)
        return y, x
    
    return inner_function

def logistic_gradient(y, x, w, h):
    
    return compute_logistic_gradient(y, x, w)
            
def logistic_error(y, x, w, h):
    
    return { 
        'logistic_err': compute_logistic_error(y, x, w),
        'n_err': compute_error_count(predict_logistic)(y, x, w)
    }

def logistic_gradient_ridge(y, x, w, h):
    
    lambda_ = h['lambda']
    
    return compute_logistic_gradient(y, x, w) + lambda_ * w

def logistic_error_and_ridge(y, x, w, h):
    
    lambda_ = h['lambda']
    
    ridge_norm = np.linalg.norm(w, 2) * lambda_
    logistic_err = compute_logistic_error(y, x, w)
    n_err = compute_error_count(predict_logistic)(y, x, w)
    
    return {
        'logistic_err': logistic_err,
        'ridge_norm': ridge_norm,
        'total_loss': logistic_err + ridge_norm,
        'n_err': n_err
    }

def logistic_gradient_lasso(y, x, w, h):
    
    lambda_ = h['lambda']
    
    return compute_logistic_gradient(y, x, w) + lambda_ * np.sign(w)

def logistic_error_and_lasso(y, x, w, h):
    
    lambda_ = h['lambda']
    
    lasso_norm = np.linalg.norm(w, 1) * lambda_
    logistic_err = compute_logistic_error(y, x, w)
    n_err = compute_error_count(predict_logistic)(y, x, w)
    
    return {
        'logistic_err': logistic_err,
        'lasso_norm': lasso_norm,
        'total_loss': logistic_err + lasso_norm,
        'n_err': n_err
    }

### Without Validation

##### Stochastic Gradient Descent

In [30]:
hs = {
    'batch_size': 2500,
    'degree': np.concatenate([[-2], np.arange(1, 7)]),
    'gamma': [1e-2, 1e-3], 
    'k_fold': 4,
    'lambda': 0,
    'max_iters': 3000,
    'num_batches': 1,
    'seed': 1,
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_descent')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = descent_with_loss(stochastic_gradient_descent_e(logistic_gradient), logistic_error),  
        round_size = 100,
        cache      = cache,
        multiple   = False,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

[-5.66090433e-02 -2.33158829e-01  1.54392859e-02  7.13685914e-02
 -2.28364642e-01  8.85929620e-02  1.81609451e-02  6.79385449e-02
 -1.57624605e-01  1.83554191e-03  3.25578471e-02  1.06221497e-01
  3.69882868e-03 -1.32188842e-01 -1.31989430e-02  1.82590860e-01
  3.42112172e-02  1.91565927e-01  4.66034936e-02  2.00314785e-02
 -9.90202102e-02 -3.59525230e-03 -5.09962015e-02  1.87760983e-02
 -7.86058461e-03  1.58727369e-02 -1.34388230e-01  3.81637454e-03
 -2.69126551e-02 -2.36458298e-02 -3.47310647e-03 -4.66701551e-03
 -2.92534988e-02  7.85894007e-02 -5.08232657e-02  9.36096724e-02
 -4.04935316e-02  8.27693349e-02 -2.12741365e-02 -5.11344342e-02
  1.56752939e-02 -6.87943374e-02  6.21099735e-02  1.95177918e-02
  7.31228713e-03  3.08135979e-02 -7.83645882e-03 -1.46074521e-02
  1.46859262e-02  3.75379822e-05 -6.87021791e-04 -1.07898604e-02
 -1.29628379e-02  7.62668440e-03  3.19266557e-02  3.40163192e-02
  2.58248548e-02 -2.32883442e-03 -1.94462854e-01 -7.10885609e-03
 -1.06259766e-02 -1.41300

iteration 600 - {'batch_size': 2500, 'degree': -2, 'gamma': 0.01, 'k_fold': 4, 'lambda': 0, 'max_iters': 100, 'num_batches': 1, 'seed': 501}
[-6.51881981e-02 -3.30378491e-01  5.59165665e-02  1.36262555e-01
 -3.20069859e-01  9.68299906e-02  1.31975797e-02  1.43599513e-01
 -2.22579683e-01 -1.86375726e-02  4.25957434e-02  1.40082303e-01
  2.25497064e-02 -1.89028502e-01 -1.15498985e-02  2.44164882e-01
  8.09398102e-02  2.82954086e-01  2.85873204e-02  2.51270695e-02
 -1.10311846e-01 -1.11464665e-02 -5.34365879e-02  4.91262694e-02
 -7.00215219e-03  3.11021061e-02 -1.77864652e-01  1.61928819e-02
 -1.22290887e-02 -5.35840223e-02 -6.84672644e-03 -2.54272504e-02
 -2.12661153e-02  9.59008440e-02 -5.08321198e-02  1.07206386e-01
 -4.80822641e-02  9.52863861e-02 -2.47481486e-02 -8.42692083e-02
 -1.76629344e-03 -1.25416851e-01  8.45862350e-02  9.78526650e-03
  1.18695469e-02  4.59823686e-02 -5.02774414e-03 -2.93502579e-02
  2.13262870e-02  3.23305582e-03 -1.31015843e-02 -1.66104914e-02
 -1.25602716e-

iteration 900 - {'batch_size': 2500, 'degree': -2, 'gamma': 0.01, 'k_fold': 4, 'lambda': 0, 'max_iters': 100, 'num_batches': 1, 'seed': 801}
[-7.80862222e-02 -3.82179413e-01  9.34509500e-02  1.88850082e-01
 -3.82124954e-01  1.02750554e-01  1.67689491e-02  1.97737524e-01
 -2.58083912e-01 -2.77731673e-02  4.31440770e-02  1.60385886e-01
  1.99068370e-02 -2.22226632e-01 -2.96076583e-03  2.83815397e-01
  1.11707887e-01  3.49092834e-01  1.44798182e-02  3.07080402e-02
 -1.09065970e-01 -6.40565424e-03 -5.47374351e-02  8.22974256e-02
  5.21080098e-03  3.96875003e-02 -2.14765494e-01  1.89513453e-02
 -9.03475311e-03 -5.82108265e-02  8.36526586e-04 -3.61021508e-02
 -2.91817284e-02  9.79335772e-02 -5.38521742e-02  1.02805265e-01
 -4.36405350e-02  9.78275972e-02 -4.22766733e-02 -1.16173171e-01
 -2.45356089e-02 -1.72066712e-01  9.05388581e-02 -1.34557332e-03
  1.00126179e-02  6.74706107e-02 -5.13478466e-03 -3.06149200e-02
  2.39011835e-02 -4.31727987e-04 -5.62752136e-03 -2.64838191e-02
 -9.10007673e-

iteration 1200 - {'batch_size': 2500, 'degree': -2, 'gamma': 0.01, 'k_fold': 4, 'lambda': 0, 'max_iters': 100, 'num_batches': 1, 'seed': 1101}
[-0.09125287 -0.42095546  0.11461867  0.22360418 -0.4425019   0.10264674
  0.00532844  0.24096732 -0.2807984  -0.03655234  0.04140979  0.17676022
  0.03655606 -0.24567716 -0.01941709  0.30417538  0.12862108  0.39161116
  0.00961089  0.04051205 -0.13317275 -0.0052075  -0.06452563  0.10503783
  0.00652148  0.05893426 -0.23754614  0.01016677  0.00786848 -0.06622024
 -0.01231342 -0.04852935 -0.0264739   0.09590801 -0.06856268  0.09904995
 -0.03763862  0.10229107 -0.04154471 -0.14297786 -0.04528345 -0.20816275
  0.09803448 -0.00986084  0.01598194  0.08743825 -0.00271951 -0.05221512
  0.03456968  0.00262719 -0.01015223 -0.01446395 -0.01041839  0.01404975
  0.0154243   0.02513753 -0.00808994  0.02775829 -0.39069938 -0.01676768
  0.01028205  0.00240804  0.02600647 -0.03614828 -0.0209891  -0.02149798
 -0.05377052 -0.00273241 -0.03385494 -0.13167169  0.00

iteration 1500 - {'batch_size': 2500, 'degree': -2, 'gamma': 0.01, 'k_fold': 4, 'lambda': 0, 'max_iters': 100, 'num_batches': 1, 'seed': 1401}
[-1.05195101e-01 -4.66231569e-01  1.24623228e-01  2.61717837e-01
 -4.89442933e-01  1.11929567e-01  6.68841642e-03  2.80072033e-01
 -3.12875880e-01 -3.83926868e-02  5.89821597e-02  1.96750979e-01
  4.22402106e-02 -2.66965454e-01 -1.72833331e-02  3.27012343e-01
  1.37234422e-01  4.34925040e-01  1.67816908e-02  3.54147215e-02
 -1.32962932e-01 -3.59627512e-03 -6.47223664e-02  1.28353880e-01
  8.17974593e-03  5.20882826e-02 -2.56761938e-01  1.21430142e-02
  7.04868776e-03 -7.15593464e-02 -1.23385391e-02 -4.91473707e-02
 -2.87041947e-02  1.03270654e-01 -7.95027333e-02  1.06845065e-01
 -5.27310285e-02  1.12036572e-01 -4.27626821e-02 -1.73644827e-01
 -5.72607731e-02 -2.43062406e-01  1.06769385e-01 -1.32058582e-02
  2.62981857e-02  1.04227342e-01 -4.30281624e-03 -5.70271674e-02
  2.56237719e-02  1.89402100e-03 -6.55203619e-03 -2.03105764e-02
 -1.78895179

...

iteration 1900 - {'batch_size': 2500, 'degree': -2, 'gamma': 0.001, 'k_fold': 4, 'lambda': 0, 'max_iters': 100, 'num_batches': 1, 'seed': 1801}
[-5.30846192e-02 -1.79386542e-01 -2.09523136e-03  5.02080027e-02
 -1.78247977e-01  7.43567165e-02  1.91440701e-02  4.34138667e-02
 -1.25703034e-01  1.45926569e-03  1.29094305e-02  8.56965435e-02
 -2.81096803e-03 -9.82367324e-02 -8.56653883e-03  1.45181675e-01
  1.42214131e-02  1.45196201e-01  4.50503026e-02  1.46472691e-02
 -8.58167649e-02 -6.26415134e-03 -4.95766074e-02  9.79093466e-03
 -1.65463805e-02  2.04016050e-02 -1.16565590e-01  4.93520904e-03
 -3.25626224e-02 -1.26178157e-02  1.04438081e-02 -3.83170181e-03
 -3.53505727e-02  6.60228397e-02 -4.50222603e-02  7.46671536e-02
 -4.13987078e-02  7.07276632e-02 -1.47138383e-02 -4.02240841e-02
  1.63715620e-02 -4.98218377e-02  4.98753998e-02  1.75764712e-02
  9.77529436e-04  2.63936619e-02 -8.85229711e-03 -1.40984887e-02
  8.62502400e-03 -7.15277912e-03 -7.66685090e-03 -7.92871381e-03
 -8.5638839

iteration 2200 - {'batch_size': 2500, 'degree': -2, 'gamma': 0.001, 'k_fold': 4, 'lambda': 0, 'max_iters': 100, 'num_batches': 1, 'seed': 2101}
[-5.39846317e-02 -1.95563790e-01  5.43568206e-03  5.74150729e-02
 -1.92253835e-01  7.78694024e-02  1.96164734e-02  5.12124103e-02
 -1.34859676e-01 -6.55952092e-04  1.68404116e-02  9.15619721e-02
 -2.79550116e-03 -1.08690103e-01 -8.00056845e-03  1.57364699e-01
  2.14105563e-02  1.60049716e-01  4.56582211e-02  1.66836670e-02
 -8.76984573e-02 -8.33242806e-03 -5.07205738e-02  1.16589876e-02
 -1.61906914e-02  2.25618673e-02 -1.23949920e-01  5.49322634e-03
 -3.16941615e-02 -1.54512058e-02  1.23364292e-02 -5.32687988e-03
 -3.34290611e-02  6.88912343e-02 -4.96030878e-02  7.80664080e-02
 -4.12859805e-02  7.42298184e-02 -1.51602204e-02 -4.48153687e-02
  1.62898430e-02 -5.73028551e-02  5.46848438e-02  1.85788048e-02
  5.93097353e-03  2.70112931e-02 -1.10087756e-02 -1.67863312e-02
  1.14290613e-02 -3.58541970e-03 -9.40609049e-03 -9.92171916e-03
 -6.8044728

iteration 2500 - {'batch_size': 2500, 'degree': -2, 'gamma': 0.001, 'k_fold': 4, 'lambda': 0, 'max_iters': 100, 'num_batches': 1, 'seed': 2401}
[-5.51091246e-02 -2.11523889e-01  9.86902920e-03  6.47133866e-02
 -2.05686506e-01  8.08357646e-02  1.94901511e-02  5.93048695e-02
 -1.43826143e-01 -3.91326061e-03  1.75957898e-02  9.76941254e-02
 -8.26718577e-04 -1.18274953e-01 -9.17484882e-03  1.69025947e-01
  2.90979602e-02  1.73447171e-01  4.56911226e-02  1.83589008e-02
 -9.16977715e-02 -8.46407702e-03 -5.00461851e-02  1.41638941e-02
 -1.27440685e-02  2.45916425e-02 -1.32096566e-01  5.51072683e-03
 -3.01873418e-02 -1.94100745e-02  9.09630572e-03 -6.39217486e-03
 -3.32935618e-02  7.27532356e-02 -5.18720107e-02  8.24983821e-02
 -4.25174916e-02  7.81135600e-02 -1.53363255e-02 -4.94590844e-02
  1.59685648e-02 -6.49973515e-02  6.01690634e-02  1.86447600e-02
  6.99334030e-03  2.87112019e-02 -1.16752360e-02 -1.76672957e-02
  1.06381654e-02 -2.79833945e-03 -9.91091346e-03 -1.07119971e-02
 -5.9481247

iteration 2800 - {'batch_size': 2500, 'degree': -2, 'gamma': 0.001, 'k_fold': 4, 'lambda': 0, 'max_iters': 100, 'num_batches': 1, 'seed': 2701}
[-5.70526241e-02 -2.25746565e-01  1.21459309e-02  7.13314862e-02
 -2.17296560e-01  8.25541951e-02  1.85880330e-02  6.79851201e-02
 -1.52335148e-01 -6.29437489e-03  2.05648790e-02  1.02220297e-01
  2.13824014e-03 -1.27679320e-01 -1.29393694e-02  1.79783280e-01
  3.51972681e-02  1.85320582e-01  4.66274710e-02  1.87696931e-02
 -9.24610891e-02 -9.25821856e-03 -5.07068759e-02  1.53246558e-02
 -1.52804573e-02  2.68114238e-02 -1.37279984e-01  5.11678729e-03
 -3.04002981e-02 -2.32517726e-02  7.09460919e-03 -7.81141467e-03
 -3.36275368e-02  7.41775029e-02 -5.31444555e-02  8.50234954e-02
 -4.33739333e-02  8.04528672e-02 -1.64535576e-02 -5.36362801e-02
  1.56430129e-02 -7.12744219e-02  6.44932284e-02  1.83172106e-02
  8.47820542e-03  3.01945089e-02 -1.31473701e-02 -1.90669911e-02
  1.17821923e-02 -2.46741617e-03 -1.03827736e-02 -1.08202939e-02
 -6.4690680

KeyboardInterrupt: 

###### Stochastic Gradient Descent With Ridge Regression

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': [-2] + np.arange(3, 4),
    'gamma': [1e-2, 1e-3], 
    'lambda': [1e-2, 1e-3],
    'k_fold': 4,
    'max_iters': 1000,
    'num_batches': 1,
    'seed': 1,
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_ridge_descent')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = stochastic_gradient_descent_e(logistic_gradient_ridge), 
        loss       = logistic_error_and_ridge, 
        round_size = 100,
        cache      = cache,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

##### Stochastic Gradient Descent With Lasso

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': [-2, 1, 2, 3, 4, 5, 6],
    'gamma': [1e-1, 1e-2, 1e-3], 
    'lambda': [1e-1, 1e-2, 1e-3],
    'k_fold': 4,
    'max_iters': 2000,
    'num_batches': 1,
    'seed': 0,
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_lasso_descent')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = stochastic_gradient_descent_e(logistic_gradient_lasso), 
        loss       = logistic_error_and_lasso, 
        round_size = 100,
        cache      = cache,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

### With Cross-Validation

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': np.concatenate([[-2], np.arange(1, 7)]),
    'gamma': [1e-2, 1e-3], 
    'k_fold': 4,
    'lambda': 0,
    'max_iters': 3000,
    'num_batches': 1,
    'seed': 0,
    'seed_cv': 0
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_descent_cross_validate')

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = cross_validate_descent(
            stochastic_gradient_descent_e(logistic_gradient), 
            logistic_error
        ),
        round_size = 100,
        cache      = cache,
        multiple   = True,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

In [ ]:
hs = {
    'batch_size': 2500,
    'degree': 3,
    'gamma': [1e-2, 1e-3], 
    'k_fold': 4,
    'lambda': 0,
    'max_iters': 1000,
    'num_batches': 1,
    'seed': 0,
    'seed_cv': 0
}

cache = Cache(CACHE_DIR + 'clean_standardize_expand_stochastic_logistic_descent_cross_validate')

def clean_expand(y, x, h):
        
    degree = int(h['degree'])

    x = remove_errors(x)
    x = remove_outliers(x)
    x = remove_nan_features(x)
    x = build_poly(x, degree)

    return y, x

_ = evaluate(
    clean = map_logistic(clean_standardize_expand), 
    fit   = descent_with_cache(
        descent    = cross_validate_descent(
            stochastic_gradient_descent_e(logistic_gradient), 
            logistic_error
        ),
        round_size = 100,
        cache      = cache,
        multiple   = True,
        log        = True
    ), 
    y     = y,
    x     = x,
    hs    = hs
)

# Split Dataset

In [14]:
def split_data(y, x):
    
    def categorize(x):
        if x[0] == -999.0:
            return 0
        else:
            return x[22] + 1
    
    categories = np.apply_along_axis(categorize, 1, x)

    xs = [x[categories == i] for i in np.arange(5)]
    ys = [y[categories == i] for i in np.arange(5)]
    
    return ys, xs

In [16]:
y_split, x_split split_data(y, x)

([array([-1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,
         -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1.,
         -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
         -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
          1., -1., -1., -1., -1., -1.,  1.,  1., -1., -1., -1., -1., -1.,
         -1., -1.,  1., -1., -1., -1.,